In [1]:
# Import necessary libraries
!pip install langchain
!pip install langchain-community
!pip install sentence-transformers
!pip install faiss-cpu

!pip install ipywidgets
import pandas as pd
import numpy as np
import torch

from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity

import ipywidgets as widgets
from IPython.display import display, HTML
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document



/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [2]:
# Load the datasets
amazon_data = pd.read_csv('amazon.csv')
amazon_data.head()

,product_id,product_name,category,discounted_price,actual_price,discount_percentage,rating,rating_count,about_product,user_id,user_name,review_id,review_title,review_content,img_link,product_link
0,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories|Accessories&Peripherals|...,₹399,"₹1,099",64%,4.2,"24,269",High Compatibility : Compatible With iPhone 12...,"AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...","Manav,Adarsh gupta,Sundeep,S.Sayeed Ahmed,jasp...","R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1K...","Satisfied,Charging is really fast,Value for mo...",Looks durable Charging is fine tooNo complains...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Wayona-Braided-WN3LG1-Sy...
1,B098NS6PVG,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,Computers&Accessories|Accessories&Peripherals|...,₹199,₹349,43%,4.0,"43,994","Compatible with all Type C enabled devices, be...","AECPFYFQVRUWC3KGNLJIOREFP5LQ,AGYYVPDD7YG7FYNBX...","ArdKn,Nirbhay kumar,Sagar Viswanathan,Asp,Plac...","RGIQEG07R9HS2,R1SMWZQ86XIN8U,R2J3Y1WL29GWDE,RY...","A Good Braided Cable for Your Type C Device,Go...",I ordered this cable to connect my phone to An...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Ambrane-Unbreakable-Char...
2,B096MSW6CT,Sounce Fast Phone Charging Cable & Data Sync U...,Computers&Accessories|Accessories&Peripherals|...,₹199,"₹1,899",90%,3.9,"7,928",【 Fast Charger& Data Sync】-With built-in safet...,"AGU3BBQ2V2DDAMOAKGFAWDDQ6QHA,AESFLDV2PT363T2AQ...","Kunal,Himanshu,viswanath,sai niharka,saqib mal...","R3J3EQQ9TZI5ZJ,R3E7WBGK7ID0KV,RWU79XKQ6I1QF,R2...","Good speed for earlier versions,Good Product,W...","Not quite durable and sturdy,https://m.media-a...",https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Sounce-iPhone-Charging-C...
3,B08HDJ86NZ,boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...,Computers&Accessories|Accessories&Peripherals|...,₹329,₹699,53%,4.2,"94,363",The boAt Deuce USB 300 2 in 1 cable is compati...,"AEWAZDZZJLQUYVOVGBEUKSLXHQ5A,AG5HTSFRRE6NL3M5S...","Omkar dhale,JD,HEMALATHA,Ajwadh a.,amar singh ...","R3EEUZKKK9J36I,R3HJVYCLYOY554,REDECAZ7AMPQC,R1...","Good product,Good one,Nice,Really nice product...","Good product,long wire,Charges good,Nice,I bou...",https://m.media-amazon.com/images/I/41V5FtEWPk...,https://www.amazon.in/Deuce-300-Resistant-Tang...
4,B08CF3B7N1,Portronics Konnect L 1.2M Fast Charging 3A 8 P...,Computers&Accessories|Accessories&Peripherals|...,₹154,₹399,61%,4.2,"16,905",[CHARGE & SYNC FUNCTION]- This cable comes wit...,"AE3Q6KSUK5P75D5HFYHCRAOLODSA,AFUGIFH5ZAFXRDSZH...","rahuls6099,Swasat Borah,Ajay Wadke,Pranali,RVK...","R1BP4L2HH9TFUP,R16PVJEXKV6QZS,R2UPDB81N66T4P,R...","As good as original,Decent,Good one for second...","Bought this instead of original apple, does th...",https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Portronics-Konnect-POR-1...


In [3]:
# Display the first few rows of each dataset to understand their structure
print("\Amazon Data Overview:")
print(amazon_data.head())


\Amazon Data Overview:
   product_id                                       product_name  \
0  B07JW9H4J1  Wayona Nylon Braided USB to Lightning Fast Cha...   
1  B098NS6PVG  Ambrane Unbreakable 60W / 3A Fast Charging 1.5...   
2  B096MSW6CT  Sounce Fast Phone Charging Cable & Data Sync U...   
3  B08HDJ86NZ  boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...   
4  B08CF3B7N1  Portronics Konnect L 1.2M Fast Charging 3A 8 P...   

                                            category discounted_price  \
0  Computers&Accessories|Accessories&Peripherals|...             ₹399   
1  Computers&Accessories|Accessories&Peripherals|...             ₹199   
2  Computers&Accessories|Accessories&Peripherals|...             ₹199   
3  Computers&Accessories|Accessories&Peripherals|...             ₹329   
4  Computers&Accessories|Accessories&Peripherals|...             ₹154   

  actual_price discount_percentage rating rating_count  \
0       ₹1,099                 64%    4.2       24,269   
1         ₹34

In [4]:
# Basic statistics to understand numerical columns
print("\nAmazon Data Statistics:")
print(amazon_data.describe())




Amazon Data Statistics:
        product_id                                       product_name  \
count         1465                                               1465   
unique        1351                                               1337   
top     B07JW9H4J1  Fire-Boltt Ninja Call Pro Plus 1.83" Smart Wat...   
freq             3                                                  5   

                                                 category discounted_price  \
count                                                1465             1465   
unique                                                211              550   
top     Computers&Accessories|Accessories&Peripherals|...             ₹199   
freq                                                  233               53   

       actual_price discount_percentage rating rating_count  \
count          1465                1465   1465         1463   
unique          449                  92     28         1143   
top            ₹999          

In [5]:
# Check for missing values
print("\nMissing Values in Amazon Data:")
print(amazon_data.isnull().sum())


Missing Values in Amazon Data:
product_id             0
product_name           0
category               0
discounted_price       0
actual_price           0
discount_percentage    0
rating                 0
rating_count           2
about_product          0
user_id                0
user_name              0
review_id              0
review_title           0
review_content         0
img_link               0
product_link           0
dtype: int64


In [6]:
amazon_data.head(2)

,product_id,product_name,category,discounted_price,actual_price,discount_percentage,rating,rating_count,about_product,user_id,user_name,review_id,review_title,review_content,img_link,product_link
0,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories|Accessories&Peripherals|...,₹399,"₹1,099",64%,4.2,"24,269",High Compatibility : Compatible With iPhone 12...,"AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...","Manav,Adarsh gupta,Sundeep,S.Sayeed Ahmed,jasp...","R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1K...","Satisfied,Charging is really fast,Value for mo...",Looks durable Charging is fine tooNo complains...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Wayona-Braided-WN3LG1-Sy...
1,B098NS6PVG,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,Computers&Accessories|Accessories&Peripherals|...,₹199,₹349,43%,4.0,"43,994","Compatible with all Type C enabled devices, be...","AECPFYFQVRUWC3KGNLJIOREFP5LQ,AGYYVPDD7YG7FYNBX...","ArdKn,Nirbhay kumar,Sagar Viswanathan,Asp,Plac...","RGIQEG07R9HS2,R1SMWZQ86XIN8U,R2J3Y1WL29GWDE,RY...","A Good Braided Cable for Your Type C Device,Go...",I ordered this cable to connect my phone to An...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Ambrane-Unbreakable-Char...


In [7]:
print(amazon_data.dtypes)

product_id             object
product_name           object
category               object
discounted_price       object
actual_price           object
discount_percentage    object
rating                 object
rating_count           object
about_product          object
user_id                object
user_name              object
review_id              object
review_title           object
review_content         object
img_link               object
product_link           object
dtype: object


## Data Processing

In [8]:
# Fill missing values with an empty string to avoid issues during tokenization
amazon_data.fillna('', inplace=True)

# Combine the product name and description into a single text column for datasets
amazon_data['text'] = amazon_data['product_name'] + ' ' + amazon_data['category'] + ' ' + amazon_data['about_product'] + ' ' + amazon_data['review_content']

# Check the new 'text' columns
print("\namazon Data Text Feature:")
print(amazon_data['text'].head())



amazon Data Text Feature:
0    Wayona Nylon Braided USB to Lightning Fast Cha...
1    Ambrane Unbreakable 60W / 3A Fast Charging 1.5...
2    Sounce Fast Phone Charging Cable & Data Sync U...
3    boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...
4    Portronics Konnect L 1.2M Fast Charging 3A 8 P...
Name: text, dtype: object


In [9]:
print(amazon_data.isnull().sum())

product_id             0
product_name           0
category               0
discounted_price       0
actual_price           0
discount_percentage    0
rating                 0
rating_count           0
about_product          0
user_id                0
user_name              0
review_id              0
review_title           0
review_content         0
img_link               0
product_link           0
text                   0
dtype: int64


## Feature Engineering with Langchain

In [10]:
# Load the tokenizer and model from Hugging Face
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

# Function to create embeddings for the text data
def get_embeddings(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).numpy()

# Apply embedding generation to datasets
amazon_data['embeddings'] = amazon_data['text'].apply(lambda x: get_embeddings(x))

# Check the embeddings
print("\nAmazon Data Embeddings (first product):")
print(amazon_data['embeddings'].iloc[0])


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(



Amazon Data Embeddings (first product):
[[-2.78378278e-01  4.58635204e-02 -2.57530883e-02 -2.90656555e-02
  -1.06368206e-01 -1.88239152e-04 -7.55039454e-02 -8.08119178e-02
  -1.42035350e-01 -4.27145176e-02  3.87473702e-01  7.27828294e-02
  -1.86094671e-01  9.28530544e-02 -7.43150637e-02  8.51891562e-02
  -4.25082259e-03 -1.92373851e-03 -6.32036924e-02  2.06559971e-01
  -3.64467129e-02 -7.82845840e-02  1.58803761e-01 -3.10221519e-02
   5.60808703e-02  6.98941275e-02  6.97385967e-02 -1.81033447e-01
  -6.78345514e-03 -1.08458593e-01 -6.26551658e-02 -4.24893126e-02
  -1.40141442e-01  8.83667916e-02  9.86080840e-02 -7.75915459e-02
   1.56258956e-01  1.84202045e-02 -1.73425358e-02 -1.19116627e-01
  -3.77390236e-02 -1.32744282e-01  1.38869765e-03 -8.64384510e-03
  -1.05242483e-01 -9.97003391e-02  4.76242825e-02  3.05765532e-02
  -1.19654834e-01  1.71920434e-02  2.97792330e-02 -2.70155491e-03
  -1.74470916e-01  1.79024413e-01  1.24933578e-01  3.78471799e-02
  -2.83196364e-02  5.67906089e-02  

## Model Selection (Cosine Similarity)

In [11]:
# Compute cosine similarity between products in the dataset
amazon_similarity = cosine_similarity(np.vstack(amazon_data['embeddings'].values))


In [12]:
# Convert amazon_data into a format LangChain can use
docs = [
    Document(
        page_content=row['product_name'],
        metadata={
            'product_id': row['product_id'],
            'product_name': row['product_name'],
            'category': row['category'],
            'discounted_price': row['discounted_price'],
            'rating': float(row['rating'].replace('|', '')) if row['rating'] != '|' else 0.0,
            'rating_count': row['rating_count'],
            'about_product':row['about_product']
        }
    )
    for _, row in amazon_data.iterrows()
]

# Initialize the HuggingFace Embeddings model
embeddings = HuggingFaceEmbeddings()

# Use FAISS to store embeddings
vectorstore = FAISS.from_documents(docs, embeddings)



<ipython-input-12-0963811c8272>:19: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings()
<ipython-input-12-0963811c8272>:19: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and 

## Building the Recommendation System

In [13]:
# Recommendation function that handles only product name (no category, no rating)
def recommend_product(query, top_n=5):
    # Use the full vectorstore for similarity search based on the product name
    results = vectorstore.similarity_search(query, k=top_n)

    # Return product details (name, price, etc.)
    recommended_products = []
    for result in results:
        recommended_products.append({
            'product_name': result.metadata['product_name'],
            #'discounted_price': result.metadata['discounted_price'],
            'category': result.metadata['category'],
            'about_product': result.metadata['about_product'],
            'rating': result.metadata['rating'],
            #'rating_count': result.metadata['rating_count'],
        })

    return recommended_products


## User Interface with Widget

In [14]:
# Create text input widget for product name (no category, no rating)
product_input = widgets.Text(description='Product Name:', placeholder='Enter a product name')

# Create a button to trigger the recommendation
button = widgets.Button(description='Recommend')

# Create an output widget to display the results
output = widgets.Output()

# Function to display recommendations when the button is clicked
def on_button_click(b):
    with output:
        # Clear the previous output
        output.clear_output()

        # Get user input
        product_name = product_input.value

        # Get recommended products using the recommendation function
        recommended_products = recommend_product(product_name)

        # Display the recommended products in a table format
        if recommended_products:
            df = pd.DataFrame(recommended_products)
            display(HTML(df.to_html(index=False)))
        else:
            print("No products found.")

# Link the button click event to the function
button.on_click(on_button_click)

# Display the widgets and output area
display(product_input, button, output)

#Bluetooth headphones

Text(value='', description='Product Name:', placeholder='Enter a product name')

Button(description='Recommend', style=ButtonStyle())

Output()